In [1]:
import pickle
from ownership import *

In [3]:
with open('data/tx_rows.p', 'rb') as f:
    tx_rows = pickle.load(f)

Let's try filtering on events instead of transactions. It's more straightforward to write a simulator for PepeDapp using *just* the CardsTransferred event!

First, I'm going to compare the keccak hash used in topics[0] for the *first* PepeDapp tx to sanity check my mechanism for finding events.

In [42]:
from Crypto.Hash import keccak
keccak_hash = keccak.new(digest_bits=256)
keccak_hash.update(b'CardCreated(bytes32,uint256)')
test_val = keccak_hash.hexdigest()
val_from_bq = '5ac4a187d29c4350cc4df530b82f63c563573f6a54125942974ca8c99e4bede9'
test_val == val_from_bq

True

In [45]:
cards_transferred_hash = keccak.new(digest_bits=256)
cards_transferred_hash.update(b'CardsTransferred(bytes32,address,address,uint256)')
'0x' + cards_transferred_hash.hexdigest()

'0xca854e9a512debe5ebf4a5228b275f9d8c3c005f95ba3660d7a6a66fdc72b112'

This is the value that I want to query with in the BigQuery event logs table. 

Querying done! Now, let's parse this data properly, particularly the other topics in each event log.

Having looked at the logs manually, it looks like:
- tx[1] == sig
- tx[2] == fromAddr
- tx[3] == toAddr
- data == count (as hex str though)

In [34]:
import json
with open('data/events.json', 'r') as f:
    logs_data = json.load(f)
    
transfers = []
for log in logs_data:
    sig = log['topics'][1]
    from_addr = Web3.toChecksumAddress('0x' + log['topics'][2][26:])
    to_addr = Web3.toChecksumAddress('0x' + log['topics'][3][26:])
    count = int(log['data'], 0)
    transfers.append(Transfer(sig, from_addr, to_addr, count))

Ok, let's go back to creating the initial state. Don't need to keep track of sales anymore!

In [13]:
sigs = get_all_sigs()
ownership = {ADMIN_ADDRESS: {}}
for sig in sigs:
    count = get_sig_count(sig)
    ownership[ADMIN_ADDRESS]['0x' + sig.hex()] = count
INITIAL_STATE = PepeState(ownership)

In [40]:
count = 0
cur_state = INITIAL_STATE
for transfer in transfers:
    cur_state = process_transfer(cur_state, transfer)
    count += 1

Hells yes! Let's create a single CSV, sorted by (sig, address) of the ownership map:

In [53]:
ownership_tuples = []
for address, sig_to_count in cur_state.ownership.items():
    for sig, count in sig_to_count.items():
        if count > 0:
            ownership_tuples.append((sig, address, count))
ownership_tuples.sort()

import csv
with open('ownership.csv', 'w') as f:
    writer = csv.writer(f)
    for t in ownership_tuples:
        writer.writerow(t)